# Converting Slippi to a DataFrame: The Frames
## Table of Contents
1. [Imports](#import)
2. [Reading in Metadata](#metadata)

<a id = 'import'></a>
## Imports

In [1]:
import pandas as pd
import numpy as np
import slippi as slp
import os

<a id = 'metadata'></a>
## Reading in MetaData

In [2]:
df_fp9 = pd.read_csv('../data/fp9.csv')
df_fp9.head()

,Unnamed: 0,game_id,date,duration,platform,p1_port,p1_char,p2_port,p2_char,stage,is_teams,is_pal
0,0,20190406T182021,2019-04-06 18:20:21+00:00,11653,Platform.NINTENDONT,0,14,3,9,32,False,False
1,1,20190406T054329,2019-04-06 05:43:29+00:00,1435,Platform.NINTENDONT,1,20,2,18,31,False,False
2,2,20190406T113710,2019-04-06 11:37:10+00:00,7577,Platform.NINTENDONT,0,22,1,2,3,True,False
3,3,20190406T060932,2019-04-06 06:09:32+00:00,9589,Platform.NINTENDONT,0,20,3,7,28,False,False
4,4,20190406T063208,2019-04-06 06:32:08+00:00,10043,Platform.NINTENDONT,0,14,3,9,8,False,False


In [3]:
# Dropping Unnamed: 0 column
df_fp9.drop(columns = ['Unnamed: 0'], inplace = True)
df_fp9.head()

,game_id,date,duration,platform,p1_port,p1_char,p2_port,p2_char,stage,is_teams,is_pal
0,20190406T182021,2019-04-06 18:20:21+00:00,11653,Platform.NINTENDONT,0,14,3,9,32,False,False
1,20190406T054329,2019-04-06 05:43:29+00:00,1435,Platform.NINTENDONT,1,20,2,18,31,False,False
2,20190406T113710,2019-04-06 11:37:10+00:00,7577,Platform.NINTENDONT,0,22,1,2,3,True,False
3,20190406T060932,2019-04-06 06:09:32+00:00,9589,Platform.NINTENDONT,0,20,3,7,28,False,False
4,20190406T063208,2019-04-06 06:32:08+00:00,10043,Platform.NINTENDONT,0,14,3,9,8,False,False


Are there any duplicate games? If there are no duplicate `game_id`s, then set `game_id` as the index. It should set the column as the index because ` copy` is in the `game_id` for those that are true duplicates.

In [4]:
if df_fp9.shape == df_fp9.drop_duplicates(subset = ['game_id']).shape:
    df_fp9.set_index('game_id', inplace = True)
    print(True)
else:
    print(False)
df_fp9.head()

True


,date,duration,platform,p1_port,p1_char,p2_port,p2_char,stage,is_teams,is_pal
game_id,,,,,,,,,,
20190406T182021,2019-04-06 18:20:21+00:00,11653,Platform.NINTENDONT,0,14,3,9,32,False,False
20190406T054329,2019-04-06 05:43:29+00:00,1435,Platform.NINTENDONT,1,20,2,18,31,False,False
20190406T113710,2019-04-06 11:37:10+00:00,7577,Platform.NINTENDONT,0,22,1,2,3,True,False
20190406T060932,2019-04-06 06:09:32+00:00,9589,Platform.NINTENDONT,0,20,3,7,28,False,False
20190406T063208,2019-04-06 06:32:08+00:00,10043,Platform.NINTENDONT,0,14,3,9,8,False,False


<a id = 'character'></a>
## Mapping Character Names to Values

In [5]:
# Reminder: Someone is Player 1 if their port index is less than the port index of the other player.
df_fp9['p1_char'].value_counts()

9     266
20    226
2     209
0     102
15     61
19     56
12     36
16     34
14     32
22     27
1      26
25     16
6      14
7      11
3       5
10      5
17      4
8       4
13      3
18      3
5       2
21      1
11      1
4       1
23      1
Name: p1_char, dtype: int64

For an easier time determining which character each player used in the game, I will be using the [documentation](https://py-slippi.readthedocs.io/en/latest/source/slippi.html) to make sure they are appropriately mapped. Upon looking into the docs, I noticed that there are two enumeration objects regarding characters, `CSSCharacter` and `InGameCharacter`. These objects label all tournament legal chracters, but in different orders. For example, Mario has a value of 0 in the `InGameCharacter` object, but has a value of 8 in `CSSCharacter`. Since I know which characters are more frequently played in tournaments, I'll take the value counts of one character column and determine if the `CSSCharacter` was interpretted or `InGameCharacter`.

| Character Value | Count |  CSSCharacter  | InGameCharacter |
|:---------------:|:-----:|:--------------:|:---------------:|
|        9        |  266  |      Marth     |      Peach      |
|        20       |  226  |      Falco     |    Young Link   |
|        2        |  209  |       Fox      |  Captain Falcon |
|        0        |  102  | Captain Falcon |      Mario      |

The most frequently used character among those I established to be Player 1 is the character whose value is 9. Since both characters are popular relative to the rest of the characters, I am not completely certain that a value of 9 represents Marth or Peach.

The second most frequent character is value 20. I am very confident that this is Falco because his performance is much better than Young Link's. So I'm inclined to say that the values represent the `CSSCharacter` object rather than the `InGameCharacgter` object. Just to be more confident than very confident, I will continue down the list.

Next is the character value of 2. This value can either represent Fox or Captain Falcon. There is not much for me to interpret because both charaters are very popular in tournament play.

The charcter value of 0 represents either Captain Falcon or Mario. This is a similar comparison to Falco and Young Link. Captain Falcon and Falco both get much more tournament play than Young Link and Mario. I can comfortably continue mapping the characters using the `CSSCharacter` values.

In [6]:
# A dictionary to map CSSCharacter values to their respoective character names
csscharacter = {
    0: 'Captain Falcon',
    1: 'Donkey Kong',
    2: 'Fox',
    3: 'Game and Watch',
    4: 'Kirby',
    5: 'Bowser',
    6: 'Link',
    7: 'Luigi',
    8: 'Mario',
    9: 'Marth',
    10: 'Mewtwo',
    11: 'Ness',
    12: 'Peach',
    13: 'Pikachu',
    14: 'Ice Climbers',
    15: 'Jigglypuff',
    16: 'Samus',
    17: 'Yoshi',
    18: 'Zelda',
    19: 'Sheik',
    20: 'Falco',
    21: 'Young Link',
    22: 'Dr. Mario',
    23: 'Roy',
    24: 'Pichu',
    25: 'Ganondorf'
}

In [7]:
# Applying the map to each appropriate column
df_fp9['p1_char_name'] = df_fp9['p1_char'].map(csscharacter)
df_fp9['p2_char_name'] = df_fp9['p2_char'].map(csscharacter)
df_fp9.head()

,date,duration,platform,p1_port,p1_char,p2_port,p2_char,stage,is_teams,is_pal,p1_char_name,p2_char_name
game_id,,,,,,,,,,,,
20190406T182021,2019-04-06 18:20:21+00:00,11653,Platform.NINTENDONT,0,14,3,9,32,False,False,Ice Climbers,Marth
20190406T054329,2019-04-06 05:43:29+00:00,1435,Platform.NINTENDONT,1,20,2,18,31,False,False,Falco,Zelda
20190406T113710,2019-04-06 11:37:10+00:00,7577,Platform.NINTENDONT,0,22,1,2,3,True,False,Dr. Mario,Fox
20190406T060932,2019-04-06 06:09:32+00:00,9589,Platform.NINTENDONT,0,20,3,7,28,False,False,Falco,Luigi
20190406T063208,2019-04-06 06:32:08+00:00,10043,Platform.NINTENDONT,0,14,3,9,8,False,False,Ice Climbers,Marth


## Mapping Stage Names to Values

In [8]:
df_fp9['stage'].value_counts()

31    278
32    205
28    186
8     175
2     151
3     149
20      1
14      1
Name: stage, dtype: int64

I know that only six stages are tournament legal, so I anticipate that stage values 20 and 14 are not tournament legal stages. The other six should be Final Destiantion, Battlefield, Yoshi's Story, Fountain of Dreams, Dream Land 64, and Pokemon Stadium. Pokemon Stadium should be either value 2 or 3 because player's cannot start on that stage unless they both consent to it.

In [9]:
stages = {
    2: 'Fountain of Dreams',
    3: 'Pokemon Stadium',
    4: "Princess Peach's Castle",
    5: 'Kongo Jungle',
    6: 'Brinstar',
    7: 'Corneria',
    8: "Yoshi's Story",
    9: 'Onett',
    10: 'Mute City',
    11: 'Rainbow Cruise',
    12: 'Jungle Japes',
    13: 'Great Bay',
    14: 'Hyrule Temple',
    15: 'Brinstar Depths',
    16: "Yoshi's Island",
    17: 'Green Greens',
    18: 'Fourside',
    19: 'Mushroom Kingdom I',
    20: 'Mushroom Kingdom II',
    22: 'Venom',
    23: 'Poke Floats',
    24: 'Big Blue',
    25: 'Icicle Mountain',
    26: 'Icetop',
    27: 'Flat Zone',
    28: 'Dream Land 64',
    29: "Yoshi's Island 64",
    30: 'Kongo Jungle 64',
    31: 'Battlefield',
    32: 'Final Destination'
}

In [10]:
df_fp9['stage_name'] = df_fp9['stage'].map(stages)
df_fp9.head()

,date,duration,platform,p1_port,p1_char,p2_port,p2_char,stage,is_teams,is_pal,p1_char_name,p2_char_name,stage_name
game_id,,,,,,,,,,,,,
20190406T182021,2019-04-06 18:20:21+00:00,11653,Platform.NINTENDONT,0,14,3,9,32,False,False,Ice Climbers,Marth,Final Destination
20190406T054329,2019-04-06 05:43:29+00:00,1435,Platform.NINTENDONT,1,20,2,18,31,False,False,Falco,Zelda,Battlefield
20190406T113710,2019-04-06 11:37:10+00:00,7577,Platform.NINTENDONT,0,22,1,2,3,True,False,Dr. Mario,Fox,Pokemon Stadium
20190406T060932,2019-04-06 06:09:32+00:00,9589,Platform.NINTENDONT,0,20,3,7,28,False,False,Falco,Luigi,Dream Land 64
20190406T063208,2019-04-06 06:32:08+00:00,10043,Platform.NINTENDONT,0,14,3,9,8,False,False,Ice Climbers,Marth,Yoshi's Story


In [11]:
df_fp9['stage'].value_counts()

31    278
32    205
28    186
8     175
2     151
3     149
20      1
14      1
Name: stage, dtype: int64

In [12]:
df_fp9['stage_name'].value_counts()

Battlefield            278
Final Destination      205
Dream Land 64          186
Yoshi's Story          175
Fountain of Dreams     151
Pokemon Stadium        149
Hyrule Temple            1
Mushroom Kingdom II      1
Name: stage_name, dtype: int64

My hypotheses about what each stage value represents was held true. 20 and 14 are tournament illegal stages and Pokemon Stadium was value 3. I'm just too good.

## Filtering Games
Now that we are able to determine which characters are in the game, we can begin filtering the games to those with...
- Team Battle set to off
- Fox vs. Falco
- On Final Destination

Final Destination was selected to be the stage of choice because it is a simple stage with no floating platforms. In next models, I will see how well the neural network can learn a player's behavior on other stages and in other matchups.

<img src="../images/final-destination.jpg" alt="Drawing" style="width: 600px;"/><center>Final Destination</center>
<img src="../images/battlefield.png" alt="Drawing" style="width: 600px;"/><center>Battlefield</center>

In [13]:
# Masks to help filter through games
mask_fd = (df_fp9['stage_name'] == 'Final Destination')
mask_fox = (df_fp9['p1_char_name'] == 'Fox') | (df_fp9['p2_char_name'] == 'Fox')
mask_falco = (df_fp9['p1_char_name'] == 'Falco') | (df_fp9['p2_char_name'] == 'Falco')
mask_teams = (df_fp9['is_teams'] == False)

In [14]:
def check_shape_frames(df0, df1):
    '''
    Prints the shape of two dataframes
    '''
    print(f'Shape of first dataframe: {df0.shape}')
    print(f'Shape of second dataframe: {df1.shape}')
    print()
    print(f'Number of frames in first dataframe: {sum(df0["duration"])}')
    print(f'Number of frames in second dataframe: {sum(df1["duration"])}')

In [15]:
# Get the rows in which Fox is either Player 1 or Player 2
df_games = df_fp9.loc[mask_fox]

check_shape_frames(df_fp9, df_games)

Shape of first dataframe: (1146, 13)
Shape of second dataframe: (432, 13)

Number of frames in first dataframe: 11255218
Number of frames in second dataframe: 3923167


In [16]:
# Get the remaining rows in which Falco is either Player 1 or Player 2
df_games = df_games.loc[mask_falco]
check_shape_frames(df_fp9, df_games)

Shape of first dataframe: (1146, 13)
Shape of second dataframe: (97, 13)

Number of frames in first dataframe: 11255218
Number of frames in second dataframe: 831522


In [17]:
# Get the remaining rows in which Team Battle is set to off
df_games = df_games.loc[mask_teams]
check_shape_frames(df_fp9, df_games)

Shape of first dataframe: (1146, 13)
Shape of second dataframe: (90, 13)

Number of frames in first dataframe: 11255218
Number of frames in second dataframe: 759580


In [18]:
# Get the remaining rows in which the players are on Final Destination
df_games = df_games.loc[mask_fd]
check_shape_frames(df_fp9, df_games)

Shape of first dataframe: (1146, 13)
Shape of second dataframe: (13, 13)

Number of frames in first dataframe: 11255218
Number of frames in second dataframe: 106178


In [19]:
# Check that the only characters remaining are Fox and Falco
print(set(df_games['p1_char_name']))
print(set(df_games['p2_char_name']))

{'Fox', 'Falco'}
{'Falco', 'Fox'}


After applying the appropriate masks to filter out the games we do not want, we have 13 games for a total of 106,178 frames. This is 0.94% of our original data, but still enough data to use as input for our model.

## Parsing Frame Data
### Getting Fox and Falco's Ports

Since I want to predict all of Fox's actions in this tournament when fighting against Falco, I will need to have a column that will specify which port is controlling Fox and which port is controlling Falco.

In [20]:
df_games.loc[df_games['p1_char_name'] == 'Fox', 'p1_port'].shape

(2,)

In [21]:
df_games.loc[df_games['p2_char_name'] == 'Fox', 'p1_port'].shape

(11,)

In [22]:
# for each game in df_games,
# if the Fox player is Player 1, append Player 1's port
# else, append Player 2's port
fox_ports = [df_games.loc[ident, 'p1_port'] if df_games.loc[ident, 'p1_char_name'] == 'Fox' else df_games.loc[ident, 'p2_port'] for ident in df_games.index]
fox_ports

[3, 3, 3, 0, 2, 3, 3, 0, 2, 2, 1, 3, 3]

In [23]:
# for each game in df_games,
# Append the port that Fox is not playing at
nfox_ports = [df_games.loc[ident, 'p1_port'] if df_games.loc[ident, 'p1_char_name'] != 'Fox' else df_games.loc[ident, 'p2_port'] for ident in df_games.index]
nfox_ports

[2, 0, 1, 3, 0, 0, 0, 1, 0, 1, 0, 2, 0]

In [24]:
# Set each list as a column to the dataframe
df_games['fox_ports'] = fox_ports
df_games['falco_ports'] = nfox_ports

In [25]:
df_games.head()

,date,duration,platform,p1_port,p1_char,p2_port,p2_char,stage,is_teams,is_pal,p1_char_name,p2_char_name,stage_name,fox_ports,falco_ports
game_id,,,,,,,,,,,,,,,
20190406T144505,2019-04-06 14:45:05+00:00,8449,Platform.NINTENDONT,2,20,3,2,32,False,False,Falco,Fox,Final Destination,3,2
20190406T190420,2019-04-06 19:04:20+00:00,7437,Platform.NINTENDONT,0,20,3,2,32,False,False,Falco,Fox,Final Destination,3,0
20190406T190322,2019-04-06 19:03:22+00:00,8206,Platform.NINTENDONT,1,20,3,2,32,False,False,Falco,Fox,Final Destination,3,1
20190309T012347,2019-03-09 01:23:47+00:00,5429,Platform.NINTENDONT,0,2,3,20,32,False,False,Fox,Falco,Final Destination,0,3
20190406T114015,2019-04-06 11:40:15+00:00,9572,Platform.NINTENDONT,0,20,2,2,32,False,False,Falco,Fox,Final Destination,2,0


### Getting the Filepaths
In the previous notebook, we used the filepaths to create the `game_id`s. Now we will work backwords and create the filepaths out of the `game_id`.

In [26]:
game_id = df_games.index
game_id

Index(['20190406T144505', '20190406T190420', '20190406T190322',
       '20190309T012347', '20190406T114015', '20190406T183745',
       '20190406T214523', '20190406T175827', '20190406T102328',
       '20190406T104203', '20190406T105900', '20190406T143503',
       '20190406T114529'],
      dtype='object', name='game_id')

In [27]:
# Use string concatenation to add the relative filepath a
# and the file extention to each value in game_id
games = list(map(lambda val: '../../data/Fight-Pitt-9/Game_' + val + '.slp', game_id))
games

['../../data/Fight-Pitt-9/Game_20190406T144505.slp',
 '../../data/Fight-Pitt-9/Game_20190406T190420.slp',
 '../../data/Fight-Pitt-9/Game_20190406T190322.slp',
 '../../data/Fight-Pitt-9/Game_20190309T012347.slp',
 '../../data/Fight-Pitt-9/Game_20190406T114015.slp',
 '../../data/Fight-Pitt-9/Game_20190406T183745.slp',
 '../../data/Fight-Pitt-9/Game_20190406T214523.slp',
 '../../data/Fight-Pitt-9/Game_20190406T175827.slp',
 '../../data/Fight-Pitt-9/Game_20190406T102328.slp',
 '../../data/Fight-Pitt-9/Game_20190406T104203.slp',
 '../../data/Fight-Pitt-9/Game_20190406T105900.slp',
 '../../data/Fight-Pitt-9/Game_20190406T143503.slp',
 '../../data/Fight-Pitt-9/Game_20190406T114529.slp']

In [28]:
def frames_to_df_fox(slp_paths):
    '''
    Returns a dataframe of frame data for each game
    slp_paths (list): A list of filepaths that lead to Slippi files
    '''
    length = len(slp_paths)
    count = 0

    # Dictionaries to keep track of which buttons on the controller we pressed for each frame
    # Fox
    fox_button_dict = {'Trigger Analog':[],'Start': [],'Y': [],'X': [],'B': [],'A': [],'L': [],'R': [],
                      'Z': [],'Dpad-Up': [],'Dpad-Down': [],'Dpad-Right': [],'Dpad-Left': []}

    # nfox = Not Fox -> Falco
    nfox_button_dict = {'Trigger Analog':[],'Start': [],'Y': [],'X': [],'B': [],'A': [],'L': [],'R': [],
                      'Z': [],'Dpad-Up': [],'Dpad-Down': [],'Dpad-Right': [],'Dpad-Left': []}
    
    # foreign key to metadata dataframe
    game_id = list()
    
    # frame index
    index = list()
    
    # feature per frame for fox
    fox_combo_count, fox_dmg, fox_direction, \
    fox_last_attack_landed, fox_last_hit_by, fox_position_x, fox_position_y, \
    fox_shield, fox_state, fox_stage_age, fox_stocks, fox_cstick_x, fox_cstick_y, fox_dmg, fox_direction, \
    fox_joystick_x, fox_joystick_y,  fox_position, fox_raw_analog_x, fox_state, fox_state_age = list(), list(), list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), \
    list(), list(), list(), list(), list(), list()
    
    # feature per frame for not fox
    nfox_combo_count, nfox_dmg, nfox_direction, \
    nfox_last_attack_landed, nfox_last_hit_by, nfox_position_x, nfox_position_y, \
    nfox_shield, nfox_state, p2_stage_age, nfox_stocks, nfox_cstick_x, nfox_cstick_y, nfox_dmg, nfox_direction, \
    nfox_joystick_x, nfox_joystick_y, nfox_position, nfox_raw_analog_x, nfox_state, nfox_state_age = list(), list(), list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), \
    list(), list(), list(), list(), list(), list()
    
    # For each filepath in the provided list of filepaths
    for path in slp_paths:

        # Create game_id
        curr_gameid = slp_paths[count].split('/')[-1].strip('Game_').strip('.slp')
        print(f'Parsing file {count + 1} of {length}')
        
        # Try to instantiate the Game object. If cannot, skip it
        try:
            game = slp.Game(path)
        except:
            print(f'Skip game {count + 1} of {length}')
            continue

        # get fox ports and non-fox ports
        fox_ports = [df_games.loc[ident, 'p1_port'] if df_games.loc[ident, 'p1_char_name'] == 'Fox' else df_games.loc[ident, 'p2_port'] for ident in df_games.index]
        nfox_ports = [df_games.loc[ident, 'p1_port'] if df_games.loc[ident, 'p1_char_name'] != 'Fox' else df_games.loc[ident, 'p2_port'] for ident in df_games.index]
        
        # for each Frame object of all frames in a specific game
        frame_length = len(game.frames)
        frame_count = 0
        for frame in game.frames:
            frame_count += 1
            print(f'Parsing frame {frame_count} of {frame_length}: {round(frame_count / frame_length * 100, 2)}%', end = '\r')
            
            # Tell me what game this frame came from
            game_id.append(curr_gameid)
            
            # Tell me the frame index
            index.append(frame.index)
            
            # Tell me the Positional X value of the cstick of each player/character
            fox_cstick_x.append(frame.ports[fox_ports[count]].leader.pre.cstick.x)
            nfox_cstick_x.append(frame.ports[nfox_ports[count]].leader.pre.cstick.x)

            # Positional Y value of the cstick of each player/character
            fox_cstick_y.append(frame.ports[fox_ports[count]].leader.pre.cstick.y)
            nfox_cstick_y.append(frame.ports[nfox_ports[count]].leader.pre.cstick.y)
            
            # Positional X value of the joystick of each player/character
            fox_joystick_x.append(frame.ports[fox_ports[count]].leader.pre.joystick.x)
            nfox_joystick_x.append(frame.ports[nfox_ports[count]].leader.pre.joystick.x)
            
            # Positional Y value of the joystick
            fox_joystick_y.append(frame.ports[fox_ports[count]].leader.pre.joystick.y)
            nfox_joystick_y.append(frame.ports[nfox_ports[count]].leader.pre.joystick.y)
            
            # Combo Count
            fox_combo_count.append(frame.ports[fox_ports[count]].leader.post.combo_count)
            nfox_combo_count.append(frame.ports[nfox_ports[count]].leader.post.combo_count)
            
            # Damage
            fox_dmg.append(frame.ports[fox_ports[count]].leader.post.damage)
            nfox_dmg.append(frame.ports[nfox_ports[count]].leader.post.damage)
            
            # Direction
            fox_direction.append(frame.ports[fox_ports[count]].leader.post.direction)
            nfox_direction.append(frame.ports[nfox_ports[count]].leader.post.direction)
            
            # Last move hit by
            fox_last_hit_by.append(frame.ports[fox_ports[count]].leader.post.last_hit_by)
            nfox_last_hit_by.append(frame.ports[nfox_ports[count]].leader.post.last_hit_by)
            
            # Character's X coordinate position
            fox_position_x.append(frame.ports[fox_ports[count]].leader.post.position.x)
            nfox_position_x.append(frame.ports[nfox_ports[count]].leader.post.position.x)
            
            # Character's Y coordinate position
            fox_position_y.append(frame.ports[fox_ports[count]].leader.post.position.y)
            nfox_position_y.append(frame.ports[nfox_ports[count]].leader.post.position.y)
            
            # Character's shield size
            fox_shield.append(frame.ports[fox_ports[count]].leader.post.shield)
            nfox_shield.append(frame.ports[nfox_ports[count]].leader.post.shield)
            
            # Characer's Action state
            fox_state.append(frame.ports[fox_ports[count]].leader.post.state)
            nfox_state.append(frame.ports[nfox_ports[count]].leader.post.state)
            
            # How long has the characer been in their current action state
            fox_state_age.append(frame.ports[fox_ports[count]].leader.post.state_age)
            nfox_state_age.append(frame.ports[nfox_ports[count]].leader.post.state_age)
            
            # Number of stocks remaining
            fox_stocks.append(frame.ports[fox_ports[count]].leader.post.stocks)
            nfox_stocks.append(frame.ports[nfox_ports[count]].leader.post.stocks)

            # Get the inputs of the Fox's controller
            fox_ins = str(frame.ports[fox_ports[count]].leader.pre.buttons.logical).split('.')[1].split('|')
            
            # For each key/button or stick on the controller
            for button in fox_button_dict:

                # if that key is in the buttons that were truly inputted this frame
                if button in fox_ins:
                    # Append True to the dictionary
                    fox_button_dict[button].append(1)
                # else, append False to the dictionary
                else:
                    fox_button_dict[button].append(0)
            
            nfox_ins = str(frame.ports[nfox_ports[count]].leader.pre.buttons.logical).split('.')[1].split('|')
            for button in nfox_button_dict:
                if button in nfox_ins:
                    nfox_button_dict[button].append(1)
                else:
                    nfox_button_dict[button].append(0)
            
        count += 1

    return pd.DataFrame({
        'game_id': game_id,
        'frame_index': index,
        
        # Fox
        'fox_cstick_x': fox_cstick_x,
        'fox_cstick_y': fox_cstick_y,
        'fox_joystick_x': fox_joystick_x,
        'fox_joystick_y': fox_joystick_y,
        'fox_trigger_analog': fox_button_dict['Trigger Analog'],
        'fox_Start': fox_button_dict['Start'],
        'fox_Y': fox_button_dict['Y'],
        'fox_X': fox_button_dict['X'],
        'fox_B': fox_button_dict['B'],
        'fox_A': fox_button_dict['A'],
        'fox_L': fox_button_dict['L'],
        'fox_R': fox_button_dict['R'],
        'fox_Z': fox_button_dict['Z'],
        'fox_Dpad_Up': fox_button_dict['Dpad-Up'],
        'fox_Dpad_Down': fox_button_dict['Dpad-Down'],
        'fox_Dpad_Right': fox_button_dict['Dpad-Right'],
        'fox_Dpad_Left': fox_button_dict['Dpad-Left'],
        'fox_combo_count': fox_combo_count,
        'fox_dmg': fox_dmg,
        'fox_direction': fox_direction,
        'fox_last_hit_by': fox_last_hit_by,
        'fox_position_x': fox_position_x,
        'fox_position_y': fox_position_y,
        'fox_shield': fox_shield,
        'fox_state': fox_state,
        'fox_state_age': fox_state_age,
        'fox_stocks': fox_stocks,
        
        # Not Fox
        'nfox_cstick_x': nfox_cstick_x,
        'nfox_cstick_y': nfox_cstick_y,
        'nfox_joystick_x': nfox_joystick_x,
        'nfox_joystick_y': nfox_joystick_y,
        'nfox_trigger_analog': nfox_button_dict['Trigger Analog'],
        'nfox_Start': nfox_button_dict['Start'],
        'nfox_Y': nfox_button_dict['Y'],
        'nfox_X': nfox_button_dict['X'],
        'nfox_B': nfox_button_dict['B'],
        'nfox_A': nfox_button_dict['A'],
        'nfox_L': nfox_button_dict['L'],
        'nfox_R': nfox_button_dict['R'],
        'nfox_Z': nfox_button_dict['Z'],
        'nfox_Dpad_Up': nfox_button_dict['Dpad-Up'],
        'nfox_Dpad_Down': nfox_button_dict['Dpad-Down'],
        'nfox_Dpad_Right': nfox_button_dict['Dpad-Right'],
        'nfox_Dpad_Left': nfox_button_dict['Dpad-Left'],
        'nfox_combo_count': nfox_combo_count,
        'nfox_dmg': nfox_dmg,
        'nfox_direction': nfox_direction,
        'nfox_last_hit_by': nfox_last_hit_by,
        'nfox_position_x': nfox_position_x,
        'nfox_position_y': nfox_position_y,
        'nfox_shield': nfox_shield,
        'nfox_state': nfox_state,
        'nfox_state_age': nfox_state_age,
        'nfox_stocks': nfox_stocks
    })

In [29]:
# Parse each Slippi game
df_frames = frames_to_df_fox(games)
df_frames.head()

Parsing file 1 of 13
Parsing file 2 of 13f 8449: 100.0%
Parsing file 3 of 13f 7437: 100.0%
Parsing file 4 of 13f 8206: 100.0%
Parsing file 5 of 13f 5429: 100.0%
Parsing file 6 of 13f 9572: 100.0%
Parsing file 7 of 13f 6712: 100.0%
Parsing file 8 of 13f 6220: 100.0%
Parsing file 9 of 13f 8705: 100.0%
Parsing file 10 of 13 9281: 100.0%
Parsing file 11 of 13 7117: 100.0%
Parsing file 12 of 13f 12435: 100.0%
Parsing file 13 of 13 7148: 100.0%


,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_trigger_analog,fox_Start,fox_Y,fox_X,...,nfox_combo_count,nfox_dmg,nfox_direction,nfox_last_hit_by,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-123,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
1,20190406T144505,-122,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
2,20190406T144505,-121,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
3,20190406T144505,-120,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
4,20190406T144505,-119,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4


## Exploring the Frames

In [30]:
# Checking for null values
df_frames.isnull().mean()

game_id                0.000000
frame_index            0.000000
fox_cstick_x           0.000000
fox_cstick_y           0.000000
fox_joystick_x         0.000000
fox_joystick_y         0.000000
fox_trigger_analog     0.000000
fox_Start              0.000000
fox_Y                  0.000000
fox_X                  0.000000
fox_B                  0.000000
fox_A                  0.000000
fox_L                  0.000000
fox_R                  0.000000
fox_Z                  0.000000
fox_Dpad_Up            0.000000
fox_Dpad_Down          0.000000
fox_Dpad_Right         0.000000
fox_Dpad_Left          0.000000
fox_combo_count        0.000000
fox_dmg                0.000000
fox_direction          0.000000
fox_last_hit_by        0.536486
fox_position_x         0.000000
fox_position_y         0.000000
fox_shield             0.000000
fox_state              0.000000
fox_state_age          0.000000
fox_stocks             0.000000
nfox_cstick_x          0.000000
nfox_cstick_y          0.000000
nfox_joy

It appears that the `last_hit_by` features has a lot of missing data. Since I want the final product of this tool to perform with a user's raw game files, I cannot expect a user to take the time to clean their game files and impute their missing values. As such, I will leave these and see how well my model does.

In [31]:
df_frames.shape[0] == sum(df_games['duration'])

True

According to user Summate from the Slippi Discord channel, each character has unique action states. This is obvious, but what was crucial to know is that character's action state values can be identical, but represent different moves. For example, Fox and Falco's action state value of 341 each represent that they will use their blaster to shoot a laser. While they are the same values and have very similar, if not the same animations, the effect of each character's laser is very different. Fox is able to shoot his blaster at a faster rate than Falco, but it deals no knockback. It is crucial to map this values to be unique. This way, our model is able to interpret them as different moves rather than the same.

In [32]:
# If the value is less than 341 or greater than 375, then it is not a unique action state to Fox
# Otherwise, it is a unique action state to Fox, so add 42 to each state to make it unique
df_frames['fox_state'] = df_frames['fox_state'].map(lambda val: val if (val < 341) or (val > 375) else val + 42)

In [33]:
# Same idea as the above cell, but add 77 to each unique action state value
df_frames['nfox_state'] = df_frames['nfox_state'].map(lambda val: val if (val < 341) or (val > 375) else val + 77)

Since we need to tell the model that how much time elapses from one row to another, we need to make the index of the dataframe to be reset. I will use video editing as an analogy to help explain why I reset the index to achieve this.

If the index is reset, then the 0th row has an index of 0 and the final row has an index of 106,177. This is similar to taking the video of all the 13 games and appending each video to each other to make one large video that is 106,178 frames long.

In [34]:
df_frames.reset_index(inplace = True, drop = True)

In [35]:
df_frames.columns

Index(['game_id', 'frame_index', 'fox_cstick_x', 'fox_cstick_y',
       'fox_joystick_x', 'fox_joystick_y', 'fox_trigger_analog', 'fox_Start',
       'fox_Y', 'fox_X', 'fox_B', 'fox_A', 'fox_L', 'fox_R', 'fox_Z',
       'fox_Dpad_Up', 'fox_Dpad_Down', 'fox_Dpad_Right', 'fox_Dpad_Left',
       'fox_combo_count', 'fox_dmg', 'fox_direction', 'fox_last_hit_by',
       'fox_position_x', 'fox_position_y', 'fox_shield', 'fox_state',
       'fox_state_age', 'fox_stocks', 'nfox_cstick_x', 'nfox_cstick_y',
       'nfox_joystick_x', 'nfox_joystick_y', 'nfox_trigger_analog',
       'nfox_Start', 'nfox_Y', 'nfox_X', 'nfox_B', 'nfox_A', 'nfox_L',
       'nfox_R', 'nfox_Z', 'nfox_Dpad_Up', 'nfox_Dpad_Down', 'nfox_Dpad_Right',
       'nfox_Dpad_Left', 'nfox_combo_count', 'nfox_dmg', 'nfox_direction',
       'nfox_last_hit_by', 'nfox_position_x', 'nfox_position_y', 'nfox_shield',
       'nfox_state', 'nfox_state_age', 'nfox_stocks'],
      dtype='object')

After checking the set of values for each button on the controller, I have found that the below buttons are all never pressed. This makes sense for Start and the Dpad buttons. If a player pauses mid-match, then they forfeit one stock. The Dpad buttons cause a character to taunt. Players don't usually taunt in a match because it usually shows bad sportsmanship and leaves their character vulnerable for a long time.

In [36]:
set(df_frames['nfox_trigger_analog'].values)

{0}

In [37]:
set(df_frames['fox_trigger_analog'].values)

{0}

In [38]:
set(df_frames['fox_Start'].values)

{0}

In [39]:
set(df_frames['nfox_Start'].values)

{0}

In [40]:
set(df_frames['nfox_Dpad_Up'].values)

{0}

In [41]:
set(df_frames['nfox_Dpad_Down'].values)

{0}

In [42]:
set(df_frames['nfox_Dpad_Left'].values)

{0}

In [43]:
set(df_frames['nfox_Dpad_Right'].values)

{0}

In [44]:
set(df_frames['fox_Dpad_Up'].values)

{0}

In [45]:
set(df_frames['fox_Dpad_Down'].values)

{0}

In [46]:
set(df_frames['fox_Dpad_Left'].values)

{0}

In [47]:
set(df_frames['fox_Dpad_Right'].values)

{0}

Since these buttons are never pushed, they are not needed to feed to the model, so we can go ahead and drop them.

In [48]:
df_frames.drop(columns = ['fox_trigger_analog', 'nfox_trigger_analog', 'fox_Start', 'nfox_Start', 
                          'fox_Dpad_Up', 'fox_Dpad_Down', 'fox_Dpad_Left', 'fox_Dpad_Right', 'nfox_Dpad_Up',
                          'nfox_Dpad_Down', 'nfox_Dpad_Left', 'nfox_Dpad_Right'], inplace = True)

In [49]:
df_frames.to_csv('../fox-falco-fd-frames.csv')